### Data preparation

Importing the libraries we will use to solve the linear programming challenge

In [1]:
import numpy as np
import pandas as pd
import setup
from docplex.mp.model import Model

Reading the xslx files with all the data of the problem

In [2]:
dfdemand = pd.read_excel('data_or_test.xlsx', sheet_name='Demand')
dfMarkup = pd.read_excel('data_or_test.xlsx', sheet_name='Markup')
dfLastmile = pd.read_excel('data_or_test.xlsx', sheet_name='Last Mile')
dfStock = pd.read_excel('data_or_test.xlsx', sheet_name='Stock')
dfIniStock = pd.read_excel('data_or_test.xlsx', sheet_name='Initial Stock')
dfTransfers = pd.read_excel('data_or_test.xlsx', sheet_name='Transfers')
dfInbound = pd.read_excel('data_or_test.xlsx', sheet_name='Inbound')
dfProduct = pd.read_excel('data_or_test.xlsx', sheet_name='Product')

Visualizing the data as a dataframe and checking if it's in the correct format 

In [3]:
dfdemand.head()

,State,Seller,Product,Week,Demand (unit),Price
0,ST1,S1,P1,W1,668,1449.19
1,ST1,S1,P1,W2,887,1983.44
2,ST1,S1,P1,W3,499,1386.48
3,ST1,S1,P1,W4,281,889.35
4,ST1,S2,P2,W1,144,2115.55


Defining the list of values in which our variables will span. 

In [4]:
list((dfTransfers["Category"]).unique())

['C1', 'C2', 'C3', 'C4', 'C5']

In [5]:
days = list(range(0,28 ))
fcenters = list((dfTransfers["FC"]).unique())
sellers = list((dfdemand["Seller"]).unique())
states = list((dfdemand["State"]).unique())
products = list((dfdemand["Product"]).unique())
categories = list((dfTransfers["Category"]).unique())

Example:

In [6]:
print(states)

['ST1', 'ST2', 'ST3', 'ST4', 'ST5', 'ST6', 'ST7', 'ST8', 'ST9', 'ST10']


# The mathematical model

### Variables

We define the CPLEX model we will be using

In [7]:
mdl = Model(name="modelEcoMole")

 When creating variables, we will use the following type format:
 B,I,C,N,S or type short names (e.g.: binary, integer, continuous, semicontinuous, semiinteger)

stockAtFC represents the amount of stock that a given fullfilment center has at a given day of a given product. Therefore it must be any non negative integer.

In [8]:
stockAtFC = mdl.var_hypercube(vartype_spec='I', seq_of_keys=[fcenters, days, products])
len(stockAtFC)

25200

transferBetweenFCs represents the amount of products that a given fullfilment center transfers to another fullfilment center at a given day of a given product. Therefore it must be any non negative integer.

In [9]:
transferBetweenFCs = mdl.var_hypercube(vartype_spec='I', seq_of_keys=[fcenters, fcenters, days, products])
len(transferBetweenFCs)

75600

In [10]:
deliveryToStateFromFC = mdl.var_hypercube(vartype_spec='I', seq_of_keys=[fcenters, states, days, products])
len(deliveryToStateFromFC)

252000

In [11]:
amountInboundAtFCProd = mdl.var_hypercube(vartype_spec='I', seq_of_keys=[fcenters, days, products])
len(amountInboundAtFCProd)

25200

### Auxiliar functions

In [12]:
def giveCategoryOfProduct(p):
    return(list(dfProduct[dfProduct["Product"] == p]["Category"])[0])

In [13]:
giveCategoryOfProduct("P2")

'C5'

In [14]:
def giveSellerOfProduct(p):
    return(list(dfIniStock[dfIniStock["Product"] == p]["Seller"])[0])

In [15]:
giveSellerOfProduct("P124")

'S24'

In [16]:
def giveCostOfInbound(s,c,fc):
    return(list(dfInbound[(dfInbound["Seller"] == s) & (dfInbound["Category"] == c) & (dfInbound["FC"] == fc) ]["Cost (per unit)"])[0])

In [17]:
giveCostOfInbound("S2","C3","FC2")

30.25

In [18]:
def giveCostOfStorage(p):
    return(list(dfStock[(dfStock["Product"] == p)]["Cost (per day and unit)"])[0])

In [19]:
giveCostOfStorage("P5")

5.99

### Restrictions

In [ ]:
InboundCosts = 0
for d in days:
    print(d)
    for fc in fcenters:
        for p in products:
            c = giveCategoryOfProduct(p)
            s = giveSellerOfProduct(p)
            costThisCSFC = giveCostOfInbound(s,c,fc)
            InboundCosts = InboundCosts + costThisCSFC * amountInboundAtFCProd[fc,d,p]

0
1
2
3


In [ ]:
StorageCost = 0
for d in days:
    print(d)
    for fc in fcenters:
        for p in products:
            costThisP = giveCostOfStorage(p)
            StorageCost = StorageCost + costThisP * stockAtFC[fc,d,p]

In [ ]:
TotalCosts = InboundCosts + StorageCost + TransfersCosts + LastMileCosts

In [ ]:
msol = mdl.solve()

In [ ]:
msol.get_value(hc[1,3,"a",2])